# 14章 テストと読みやすさ

## 14.1 テストを読みやすくて保守しやすいものにする

> 他のプログラマが安心してテストの追加や変更ができるように、テストコードを読みやすくする。

テストコードといえど甘く見てはいけない。

テストコードというものは`本物のコードの動作と使い方を示した非公式な文書`と考えることができる。

## 14.3 テストを読みやすくする

一般的な設計原則として、`大切でない詳細はユーザーから隠蔽し、大切なものは目立たせる`という原則がある。

テストに関係のないコードは、関数で丸めるなどして、可能な限り隠蔽するのがベスト。

### 最小のテストを作る

テストの本質を、関数でまるめ、テストシーケンスの中で1行にまとめることができたら最高だ。
大抵のテストは、「ある入力のもと、このような出力を期待する」といったことに尽きる。
ので、テストする機能の呼び出しや、値の前処理などは、関数にまとめて、再利用できるのがいいだろう。

上のようにすることで、テストケースの追加や編集が容易になる。

### 独自の「ミニ言語」を実装する

入力値のパース処理などは、テストの本質からは外れるので、関数でまとめて、テストシーケンスの関数内から呼び出すのが効率的だ。

```TypeScript
function parseInput(input: string): number[] {
  return input.split(',').map((s) => parseInt(s, 10));
}

function checkValue(value: number): boolean {
  return value >= 0 && value <= 100;
}

const testSequence = (input: string): boolean => {
    const values = parseInput(input);
    return values.every(checkValue);
};

const test = (): void => {
    const inputs = [
        '0,1,2,3,4,5,6,7,8,9',
        '0,1,2,3,4,5,6,7,8,9,100',
        '0,1,2,3,4,5,6,7,8,9,101',
        '0,1,2,3,4,5,6,7,8,9,100,101',
    ];
    inputs.forEach((input) => {
        console.log(testSequence(input));
    });
};
```



## 14.4 エラーメッセージを読みやすくする。

多くの言語では、テスト結果と期待する結果を比較するために、`assert`関数を提供している。

```cpp
assert(1 == 2);
assert(1 == 1);
//error: assert failed: 1 == 2
```
しかし、標準ライブラリのassert関数は、エラーメッセージが不十分である。
そこで、Boost C++ライブラリを使うことで、より詳細なテスト結果を得ることができる。
```cpp
BOOST_REQUIRE_EQUAL(1, 2);
BOOST_REQUIRE_EQUAL(1, 1);
//error: test.cpp(3): fatal error in "test": check 1 == 2 failed [1 != 2]
```

そして、それでも不十分な場合は自身で作ってしまえばいい。

ここから学べることは、`エラーメッセージは詳細に書くこと`である。


## 14.5 テストの適切な入力値を選択する

テストで使う入力値は、
* 簡潔でかつ読みやすいもの
* コードを完全にテストできるもの
上の2点を満たすような値でなければいけない。

> コードを完全にテストする最も単純な入力値の組み合わせを選択しなければいけない。

例えば、`SortAndFilterDocs()`に負の値を削除するという機能がある場合、以下のようにテストコードを書いた場合は無意味だ。
```cpp
BOOST_REQUIRE_EQUAL(SortAndFilterDocs({0, 1, 2, 3}));
//BAD: 負の値が含まれていない、かつ、もうすでにソートされている。
//以下のような複雑な値もＮＧだ
BOOST_REQUIRE_EQUAL(SortAndFilterDocs({2444435345, -34531, 3452, 4533, -456671, -2, -63677}));
```
> テストには最もきれいで単純な値を使う。
